In [ ]:
import pandas as pd

food_data = pd.read_csv(r'C:\Users\antho\Documents\Projets\Hackathon\UdeM_Hack\data\processed_recipes_with_categories.csv')

In [32]:
## print les informations sur le dataframe

print(food_data.info())
print(food_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               231636 non-null  object 
 1   id                 231637 non-null  int64  
 2   minutes            231637 non-null  int64  
 3   contributor_id     231637 non-null  int64  
 4   submitted          231637 non-null  object 
 5   tags               231637 non-null  object 
 6   nutrition          231637 non-null  object 
 7   n_steps            231637 non-null  int64  
 8   steps              231637 non-null  object 
 9   description        226658 non-null  object 
 10  ingredients        231637 non-null  object 
 11  n_ingredients      231637 non-null  int64  
 12  calories           231637 non-null  float64
 13  total_fat_PDV      231637 non-null  float64
 14  sugar_PDV          231637 non-null  float64
 15  sodium_PDV         231637 non-null  float64
 16  pr

In [33]:
from bs4 import BeautifulSoup
import requests

def get_primary_image_url(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Chercher la div contenant l'image principale
    primary_image_div = soup.find('div', class_='primary-image')
    if primary_image_div:
        # Trouver la première balise <img> dans cette div
        img_tag = primary_image_div.find('img')
        if img_tag and 'src' in img_tag.attrs:
            return img_tag['src']
    
    return None

from PIL import Image
from io import BytesIO

def get_food_image_url(food_id):
    # Trouver le nom de l'aliment correspondant à l'ID
    food_name = food_data.loc[food_data['id'] == food_id, 'name'].values[0]
    
    # Construire l'URL de la page de la recette
    url = "https://www.food.com/recipe/" + str(food_name).replace(" ", "-") + "-" + str(food_id)
    response = requests.get(url)
    
    if response.status_code == 200:
        # Extraire l'URL de l'image principale
        image_url = get_primary_image_url(response.text)
        return image_url
    else:
        return None

def show_food_image(food_id):
    image_url = get_food_image_url(food_id)
    if image_url:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img.show()
    else:
        print("Impossible de récupérer l'image pour l'ID:", food_id)

# Exemple d'utilisation
food_id = 38
show_food_image(food_id)


In [35]:
def propose_recipes(ingredients_list):
    # Convertir les ingrédients en minuscules pour correspondre aux données
    ingredients_list = [ingredient.lower() for ingredient in ingredients_list]
    
    # Fonction pour compter le nombre d'ingrédients correspondants dans une recette
    def count_matching_ingredients(recipe_ingredients):
        return sum(1 for ingredient in ingredients_list if ingredient in recipe_ingredients)
    
    # Filtrer les recettes qui contiennent au moins deux des ingrédients spécifiés
    matching_recipes = food_data[food_data['ingredients'].apply(lambda x: count_matching_ingredients(x.lower())) >= 3]
    
    return matching_recipes

# Exemple d'utilisation
ingredients_list = ['tomato', 'onion', 'garlic']
matching_recipes = propose_recipes(ingredients_list).head(10)
matching_recipes[['name', 'ingredients']]

,name,ingredients
154,easy tomato bean soup,"['vegetable oil cooking spray', 'onion', 'jala..."
191,lemon chili chicken with penne,"['penne', 'boneless skinless chicken breast ha..."
277,my homemade refried beans kid pleasing qui...,"['black beans', 'pinto beans', 'vegetable oil'..."
321,ultimate flu busting chicken soup,"['chicken breasts', 'celery', 'garlic cloves',..."
338,fat free white bean dip,"['white beans', 'tomato paste', 'water', 'garl..."
355,greek rotini pasta ww friendly,"['whole wheat rotini', 'red bell pepper', 'gre..."
409,black bean cous cous salad,"['couscous', 'chicken broth', 'extra virgin ol..."
429,edamame and corn salsa,"['edamame', 'corn', 'roma tomatoes', 'cilantro..."
434,italian turkey burgers reduced fat,"['ground turkey breast', 'onion', 'fresh basil..."
460,keema mattar,"['onion', 'ginger', 'garlic paste', 'vegetable..."
